<a href="https://colab.research.google.com/github/Darren-chan-jr/Financial-Advisor-Matching-Model---Datathon-Project/blob/main/CAT_A_63.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Listing libraries that are installed
%pip list

In [ ]:
# Installing necessary libraries using pip tool
%pip install scikit-learn
%pip install pandas
%pip install numpy
%pip install matplotlib
%pip install seaborn
%pip install scipy
%pip install pyarrow
%pip install shap
%pip install lightgbm
%pip install imbalanced-learn

## Import necessary **libraries**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pyarrow.parquet as pq
from sklearn.preprocessing import OneHotEncoder, MultiLabelBinarizer, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import accuracy_score, f1_score, classification_report, silhouette_score
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
import scipy.cluster.hierarchy as sch
import shap
import re
import ast # To safely convert string representations of lists
from imblearn.over_sampling import SMOTE



## Connecting to Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Read data

In [ ]:
# @title Read Agent

AGENT_PATH = '/content/drive/MyDrive/data/final_nus_datathon_dataset/nus_agent_info_df.parquet' # To update acording to the path of your data file
agent_df = pd.read_parquet(AGENT_PATH, engine='pyarrow')
agent_df.shape
agent_df.info() # Only agent_age has 12 null values
agent_df.head()

In [ ]:
# @title Read Policy

POLICY_PATH = '/content/drive/MyDrive/data/final_nus_datathon_dataset/nus_policy_info_df.parquet' # To update acording to the path of your data file
policy_df = pd.read_parquet(POLICY_PATH, engine='pyarrow')
policy_df.shape
policy_df.info() # No null values
policy_df.head()

In [ ]:
# @title Read Client

CLIENT_PATH = '/content/drive/MyDrive/data/final_nus_datathon_dataset/nus_client_info_df.parquet' # To update acording to the path of your data file
client_df = pd.read_parquet(CLIENT_PATH, engine='pyarrow')
client_df.shape
client_df.info()
client_df.head()
# cltdob: 6 null; race_desc_map: 10 null; cltpcode: 156 null; household_size, economic_status, family_size: 343 null each

# **Exploratory Data Analysis (EDA)**

## Data Cleaning



In [ ]:
# @title Clean Agent Data

def clean_expertise(entry):
    """Cleans 'agent_product_expertise' column, ensuring lists are properly formatted."""
    if isinstance(entry, str):
        try:
            parsed_list = ast.literal_eval(entry)  # Convert string to list
            if isinstance(parsed_list, str):
                parsed_list = re.sub(r'(?<=prod_\d)(?=prod_)', ',', parsed_list)
                return parsed_list.split(',')
            return parsed_list
        except (ValueError, SyntaxError):
            return []
    elif isinstance(entry, list):
        return entry if len(entry) != 1 or not isinstance(entry[0], list) else entry[0]
    return []

# Apply cleaning
agent_df["agent_product_expertise"] = agent_df["agent_product_expertise"].apply(clean_expertise)

# Impute agent_age null values with the median age
agent_df["agent_age"] = agent_df["agent_age"].fillna(agent_df["agent_age"].median())



In [ ]:
agent_df.head()
agent_df.info()

In [ ]:
# @title Clean Policy Data

# Standardize product, age and tenure groups
def extract_number(pattern, x):
    match = re.search(pattern, x) if pd.notnull(x) else None
    return int(match.group(1)) if match else None

policy_df["product_num"] = policy_df["product"].apply(lambda x: extract_number(r'prod_(\d+)', x))
policy_df["product_grp_num"] = policy_df["product_grp"].apply(lambda x: extract_number(r'PG:(\d+)', x))
policy_df["cust_age_group_num"] = policy_df["cust_age_at_purchase_grp"].apply(lambda x: extract_number(r'AG(\d+)', x))
policy_df["cust_tenure_group_num"] = policy_df["cust_tenure_at_purchase_grp"].apply(lambda x: extract_number(r'TNR(\d+)', x))

# Drop columns
policy_df.drop(columns=["product", "product_grp", "cust_age_at_purchase_grp", "cust_tenure_at_purchase_grp", "flg_main", "flg_inforce", "flg_cancel", "flg_expire", "flg_converted"], inplace=True)


In [ ]:
policy_df.head()
policy_df.info()

In [ ]:
# @title Clean Client Data

# cltdob: 6 null; race_desc_map: 10 null; cltpcode: 156 null; household_size, economic_status, family_size: 343 null each

# Impute `cltdob` (Date of Birth) using median age
client_df["cltdob"] = pd.to_datetime(client_df["cltdob"], errors='coerce')  # Ensure datetime format
median_age = client_df["cltdob"].dropna().dt.year.median()
client_df["cltdob"] = client_df["cltdob"].fillna(pd.Timestamp(year=int(median_age), month=1, day=1))

#Converting dob to age
client_df['clt_Age'] = (pd.Timestamp.today() - client_df['cltdob']).dt.days // 365
# Convert Age to integer
client_df['clt_Age'] = client_df['clt_Age'].astype(int)

# Impute `race_desc_map` (Race) using mode ("Chinese")
race_mode = client_df["race_desc_map"].mode()[0]
client_df["race_desc_map"] = client_df["race_desc_map"].fillna(race_mode)

# Note: household_size and family_size values are strangely large. Why?
# Impute `household_size`, `economic_status`, `family_size` using mode
for col in ["household_size", "economic_status", "family_size"]:
    mode_value = client_df[col].mode()[0]
    client_df[col] = client_df[col].fillna(mode_value)


# Standardize household_size_grp and family_size_grp
client_df["household_size_grp_num"] = client_df["household_size_grp"].apply(lambda x: extract_number(r'HH(\d+)', x))
client_df["family_size_grp_num"] = client_df["family_size_grp"].apply(lambda x: extract_number(r'FS(\d+)', x))

# Drop original categorical columns after transformation
client_df.drop(columns=["household_size_grp", "family_size_grp"], inplace=True)

client_df['economic_status'] = client_df['economic_status'].astype('int')
client_df['household_size'] = client_df['household_size'].astype('int')
client_df['family_size'] = client_df['family_size'].astype('int')


In [ ]:
client_df.head()
client_df.info()

# Data Preprocessing

## Joining Agent, Policy and Client Tables

In [ ]:
# Merge policy table with client information based on 'secuityno'
policy_client_df = policy_df.merge(client_df, on="secuityno", how="left")
# Merge the result with agent information based on 'agntnum'
full_data = policy_client_df.merge(agent_df, on="agntnum", how="left")

In [ ]:
full_data.head()
full_data.info()

# One-hot Encoding for Nominal data

In [ ]:
# Categorical columns to encode
categorical_cols = ["agent_gender", "agent_marital", "cltsex", "marryd", "race_desc_map"]

# Apply OneHotEncoder
encoder = OneHotEncoder(drop='first', sparse_output=False)
encoded_array = encoder.fit_transform(full_data[categorical_cols])

# Convert to DataFrame
encoded_df = pd.DataFrame(encoded_array, columns=encoder.get_feature_names_out(categorical_cols)).astype(int)

# Merge encoded data and drop original categorical columns
full_data = pd.concat([full_data.drop(columns=categorical_cols), encoded_df], axis=1)

In [ ]:
print(full_data.head())
print(full_data.info())

In [ ]:
# One-Hot Encode Multi-Label Columns
mlb = MultiLabelBinarizer()
expertise_encoded = mlb.fit_transform(full_data['agent_product_expertise'])

# Convert to DataFrame and merge
expertise_df = pd.DataFrame(expertise_encoded, columns=mlb.classes_)
full_data = pd.concat([full_data.drop(columns=['agent_product_expertise']), expertise_df], axis=1)

In [ ]:
# @title Handle Missing Values & Final Checks

# Drop rows with remaining NaN values
full_data.dropna(inplace=True)

# Verify no missing values remain
assert full_data.isnull().sum().sum() == 0

In [ ]:
# Display cleaned dataset
print(full_data.head())

## Feature Scaling

For Client

In [ ]:
# Select numerical features
features = ['cltsex_F', 'cltsex_M', 'marryd_D', 'marryd_M', 'marryd_P', 'marryd_S', 'marryd_U', 'marryd_W',  'race_desc_map_Indian', 'race_desc_map_Malay', 'race_desc_map_Others', 'clt_Age', 'household_size', 'economic_status', 'family_size']

# Standardize features
scaler = StandardScaler()
df_scaled = scaler.fit_transform(full_data[features])

# Model Training with scikit-learn

# K-Means Clustering

In [ ]:
####

# Visualize clusters
#sns.pairplot(full_data, hue='Cluster', palette='viridis')
#plt.show()

# Silhouette Score
silhouette_scores = []

for k in range(2, 11):  # Silhouette Score requires at least 2 clusters
    kmeans = KMeans(n_clusters=k, random_state=42)
    cluster_labels = kmeans.fit_predict(df_scaled)
    silhouette_scores.append(silhouette_score(df_scaled, cluster_labels))

# Plot Silhouette Scores
plt.plot(range(2, 11), silhouette_scores, marker="o")
plt.xlabel("Number of Clusters (k)")
plt.ylabel("Silhouette Score")
plt.title("Silhouette Score for Optimal k")
plt.show()

# Fit K-Means with chosen k
optimal_k = 6
kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
full_data['Cluster'] = kmeans.fit_predict(df_scaled)


In [ ]:
print(full_data.info())

In [ ]:
# Group by cluster and analyze feature means
cluster_summary = full_data.groupby("Cluster").mean(numeric_only=True)
print(cluster_summary)

numeric_features = ['household_size', 'economic_status', 'family_size', 'clt_Age']

for feature in numeric_features:
    plt.figure(figsize=(8, 5))
    sns.boxplot(x="Cluster", y=feature, data=full_data)
    plt.title(f"{feature} by Cluster")
    plt.show()


In [ ]:
categorical_features = ['cltsex_F', 'cltsex_M', 'marryd_D', 'marryd_M', 'marryd_P', 'marryd_S', 'marryd_U', 'marryd_W',  'race_desc_map_Indian', 'race_desc_map_Malay', 'race_desc_map_Others']


# Compute the mean of each dummy variable per cluster
cluster_composition = full_data.groupby("Cluster")[categorical_features].mean()

# Plot stacked bar chart
cluster_composition.T.plot(kind="bar", stacked=True, figsize=(10, 6), colormap="viridis")
plt.title("Cluster Composition Across Categorical Variables")
plt.ylabel("Proportion of Members in Each Cluster")
plt.legend(title="Cluster")
plt.show()


In [ ]:

for feature in categorical_features:
    plt.figure(figsize=(8, 5))
    sns.countplot(x="Cluster", hue=feature, data=full_data)
    plt.title(f"Distribution of {feature} in Each Cluster")
    plt.legend(title=feature)
    plt.show()


In [ ]:
# Reduce data to 2D using PCA
pca = PCA(n_components=2)
df_pca = pca.fit_transform(df_scaled)

# Add cluster labels to the reduced data
df_pca = pd.DataFrame(df_pca, columns=["PC1", "PC2"])
df_pca["Cluster"] = full_data["Cluster"]

# Plot the clusters
sns.scatterplot(x="PC1", y="PC2", hue="Cluster", data=df_pca, palette="viridis")
plt.title("K-Means Clustering (2D PCA)")
plt.show()

# Hierarchical Clustering

In [ ]:

# Compute the linkage matrix (Ward's method minimizes variance within clusters)
linkage_matrix = sch.linkage(df_scaled, method='ward')

# Plot the Dendrogram
plt.figure(figsize=(12, 6))
sch.dendrogram(linkage_matrix, truncate_mode='level', p=10)  # Show only top 10 levels
plt.xlabel("Data Points")
plt.ylabel("Distance")
plt.title("Hierarchical Clustering Dendrogram")
plt.show()

In [ ]:
# @title Dont run this!!!!! (Very time consuming)

silhouette_scores = []

# Test different cluster numbers
for k in range(2, 11):  # Hierarchical clustering must have at least 2 clusters
    hierarchical = AgglomerativeClustering(n_clusters=k, linkage="ward")
    cluster_labels = hierarchical.fit_predict(df_scaled)

    score = silhouette_score(df_scaled, cluster_labels)
    silhouette_scores.append(score)

# Plot Silhouette Scores
plt.figure(figsize=(8, 5))
plt.plot(range(2, 11), silhouette_scores, marker="o", linestyle="-", color="b")
plt.xlabel("Number of Clusters (k)")
plt.ylabel("Silhouette Score")
plt.title("Silhouette Score for Different Cluster Counts (Hierarchical Clustering)")
plt.grid(True)
plt.show()


In [ ]:

agglomerative = AgglomerativeClustering(n_clusters=6)
full_data["Cluster_Hierarchical"] = agglomerative.fit_predict(df_scaled)


In [ ]:
silhouette_hc = silhouette_score(df_scaled, full_data["Cluster_Hierarchical"])
print(f"Silhouette Score (Hierarchical): {silhouette_hc}")

In [ ]:
# Define the mapping of numerical cluster labels to descriptive names
cluster_mapping = {
    0: "Affluent Middle Aged Chinese Families with Mixed Marital Status",
    1: "Middle-Aged Married Chinese and Multiracial Families",
    2: "Middle-Aged Divorced Individuals with Mixed Ethnicity and Small Families",
    3: "Middle-Aged Married Individuals with Mixed Ethnicity and Large Families",
    4: "Young Separated Chinese Individuals with Small Families",
    5: "Elderly Widowed Chinese Women with Low Economic Status"
}

# Apply the mapping to the cluster labels
full_data["Cluster_Hierarchical"] = full_data["Cluster_Hierarchical"].replace(cluster_mapping)

# Display the updated DataFrame with new labels
print(full_data[["Cluster_Hierarchical"]].head())


# Descriptive Analysis of Cluster Characteristics

In [ ]:

# Convert specific columns to numeric if they are expected to be numeric
for col in full_data.columns:
    if full_data[col].dtype == 'object':  # Check if the column is of type 'object'
        try:
            full_data[col] = pd.to_numeric(full_data[col], errors='ignore') # Try converting to numeric; ignore errors if conversion fails
        except (ValueError, TypeError):
            pass  # Handle potential errors during conversion, e.g., if the column contains non-numeric strings

# Calculate cluster summary using the modified DataFrame
cluster_summary = full_data.groupby("Cluster_Hierarchical").mean(numeric_only=True)  #Explicitly tell pandas to only consider numeric columns for the mean calculation.
print(cluster_summary)


In [ ]:
numeric_features = ['household_size', 'economic_status', 'family_size', 'clt_Age']

for feature in numeric_features:
    plt.figure(figsize=(8, 5))
    sns.boxplot(x="Cluster_Hierarchical", y=feature, data=full_data)
    plt.title(f"{feature} by Cluster")
    plt.xticks(rotation=90)
    plt.show()

In [ ]:
categorical_features = ['cltsex_F', 'cltsex_M', 'marryd_D', 'marryd_M', 'marryd_P', 'marryd_S', 'marryd_U', 'marryd_W',  'race_desc_map_Indian', 'race_desc_map_Malay', 'race_desc_map_Others']
for feature in categorical_features:
    plt.figure(figsize=(8, 5))
    sns.countplot(x="Cluster_Hierarchical", hue=feature, data=full_data)
    plt.title(f"Distribution of {feature} in Each Cluster")
    plt.legend(title=feature)
    plt.xticks(rotation=90)
    plt.show()


## Identifying Agent Feature Importance for each cluster



In [ ]:
#%pip install pandas
#%pip install scikit-learn
%pip install imbalanced-learn
%pip install lightgbm

import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report, accuracy_score

# Define features (independent variables)
agent_features = [
    "agent_age", "agent_tenure", "cnt_converted", "annual_premium_cnvrt",
    "pct_lapsed", "pct_cancel", "pct_inforce",
    "pct_prod_0_cnvrt", "pct_prod_1_cnvrt", "pct_prod_2_cnvrt", "pct_prod_3_cnvrt",
    "pct_prod_4_cnvrt", "pct_prod_5_cnvrt", "pct_prod_6_cnvrt", "pct_prod_7_cnvrt",
    "pct_prod_8_cnvrt", "pct_prod_9_cnvrt",
    "pct_SX0_unknown", "pct_SX1_male", "pct_SX2_female",
    "pct_AG01_lt20", "pct_AG02_20to24", "pct_AG03_25to29", "pct_AG04_30to34",
    "pct_AG05_35to39", "pct_AG06_40to44", "pct_AG07_45to49", "pct_AG08_50to54",
    "pct_AG09_55to59", "pct_AG10_60up",
    "agent_gender_M", "agent_gender_U",
    "agent_marital_M", "agent_marital_S", "agent_marital_U", "agent_marital_W"
]

X = full_data[agent_features]  # Features
y = full_data["Cluster"]  # Target (customer cluster)

# Dictionary to store trained models for each cluster
models_per_cluster = {}

# Loop through each unique cluster and train a separate model
for cluster in y.unique():
    print(f"Training model for Cluster {cluster}...")

    # Select only agents assigned to this cluster
    X_cluster = X[y == cluster]
    y_cluster = y[y == cluster]



    # Split into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(
        X_cluster, y_cluster, test_size=0.2, random_state=42
    )

    # Train LightGBM model for this cluster
    model = lgb.LGBMClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    # Store model in dictionary
    models_per_cluster[cluster] = model

    # Evaluate model
    y_pred = model.predict(X_test)
    print(f"Cluster {cluster} Accuracy: {accuracy_score(y_test, y_pred)}")
    print(f"Cluster {cluster} Classification Report:\n{classification_report(y_test, y_pred)}\n")

# Display trained models
print("Models trained for clusters:", models_per_cluster.keys())

In [ ]:
# @title Gini Importance
# feature_importance = pd.Series(rf_model.feature_importances_, index=X.columns).sort_values(ascending=False)
# print(feature_importance)

feature_names = X.columns
importances = rf_model.feature_importances_
feature_impt = pd.DataFrame({'Feature': feature_names, 'Gini Importance': importances}).sort_values('Gini Importance', ascending=False)
print(feature_impt)


In [ ]:
# Select top 15 features based on Gini Importance
top_n = 15
top_features_df = feature_impt.nlargest(top_n, 'Gini Importance')

# Create a horizontal bar plot for the top 15 features
plt.figure(figsize=(10, 6))
plt.barh(top_features_df["Feature"], top_features_df["Gini Importance"], color='skyblue')

# Formatting
plt.xlabel('Gini Importance')
plt.ylabel('Feature')
plt.title(f'Top {top_n} Most Important Features - Gini Importance')
plt.gca().invert_yaxis()  # Flip Y-axis for readability

# Show plot
plt.show()

In [ ]:
# @title SHAP Values

# Create SHAP Explainer
explainer = shap.TreeExplainer(rf_model)

# Compute SHAP values
shap_values = explainer.shap_values(X_test)  # Get SHAP values for test data

# Check the shape of SHAP values
print(np.array(shap_values).shape)  # Should be (num_classes, num_samples, num_features)

# Summary Plot
shap.summary_plot(shap_values, X_test)

# Feature Importance Bar Chart
shap.summary_plot(shap_values, X_test, plot_type="bar")

In [ ]:
# @title Hyperparameter Tuning

# Define hyperparameter grid
param_dist = {
    "n_estimators": [100, 200, 500, 1000],
    "max_depth": [10, 20, 30, None],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4],
    "max_features": ["sqrt", "log2", None]
}

# Randomized Search
rf_random = RandomizedSearchCV(
    estimator=RandomForestClassifier(random_state=42),
    param_distributions=param_dist,
    n_iter=20,  # Number of different combinations to try
    cv=3,  # 3-fold cross-validation
    verbose=2,
    n_jobs=-1,
    random_state=42
)

rf_random.fit(X_train, y_train)

# Best parameters
print("Best Parameters:", rf_random.best_params_)


In [ ]:
# Train model with best parameters
final_model = RandomForestClassifier(rf_random.best_params_, random_state=42)
final_model.fit(X_train, y_train)

# Predictions
y_pred_final = final_model.predict(X_test)

# Final Model Accuracy
print("Final Model Accuracy:", accuracy_score(y_test, y_pred_final))
print(classification_report(y_test, y_pred_final))


# Model Evaluation

In [ ]:
# Merge client clusters with agent features
recommendation_data = full_data[["Cluster_Hierarchical", "agntnum"] + top_agent_features]

# Define features (X) and target (y)
X = recommendation_data.drop(columns=["agntnum"])  # Features: client cluster + agent features
y = recommendation_data["agntnum"]  # Target: best agent

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Random Forest Model
rf_optimized = RandomForestClassifier(n_estimators=100, random_state=42)
rf_optimized.fit(X_train, y_train)

# Predictions
y_pred = rf_optimized.predict(X_test)

# Evaluate Performance
from sklearn.metrics import accuracy_score, classification_report

print("Optimized Model Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


In [ ]:
# @title Assign Customers to an Agent Cluster

# Select agent performance features
agent_performance_features = ["cnt_converted", "annual_premium_cnvrt", "pct_lapsed", "pct_cancel", "pct_inforce"]

# Apply K-Means to group agents into clusters
kmeans_agents = KMeans(n_clusters=4, random_state=42)
full_data["agent_cluster"] = kmeans_agents.fit_predict(full_data[agent_performance_features])

# Map customers to agent clusters
full_data["recommended_agent_cluster"] = full_data["Cluster_Hierarchical"].map(
    lambda x: kmeans_agents.predict(
        full_data[full_data["Cluster_Hierarchical"] == x][agent_performance_features].mean().values.reshape(1, -1)
    )[0]
)



In [ ]:
# Step 1: Group by Cluster and Agent
agent_cluster_metrics = full_data.groupby(["Cluster_Hierarchical", "agntnum"]).agg(
    cnt_converted=("cnt_converted", "sum"),  # Total policies converted
    pct_lapsed=("pct_lapsed", "mean"),      # Average lapse percentage
    pct_cancel=("pct_cancel", "mean"),      # Average cancel percentage
    pct_inforce=("pct_inforce", "mean")     # Average in-force percentage
).reset_index()

# Step 2: Calculate Normalized Success Rate
# Adjust for the balance between in-force, lapsed, and canceled policies
agent_cluster_metrics["normalized_success_rate"] = (
    agent_cluster_metrics["pct_inforce"]
    - agent_cluster_metrics["pct_lapsed"]
    - agent_cluster_metrics["pct_cancel"]
)

# Normalize to a scale (0 to 1)
agent_cluster_metrics["normalized_success_rate"] = (
    agent_cluster_metrics["normalized_success_rate"]
    - agent_cluster_metrics["normalized_success_rate"].min()
) / (
    agent_cluster_metrics["normalized_success_rate"].max()
    - agent_cluster_metrics["normalized_success_rate"].min()
)

# Step 3: Combine Metrics for Final Ranking
agent_cluster_metrics["combined_score"] = (
    agent_cluster_metrics["cnt_converted"] * 0.6  # Weight for count converted
    + agent_cluster_metrics["normalized_success_rate"] * 0.4  # Weight for success rate
)

# Rank agents within each cluster
agent_cluster_metrics["rank"] = agent_cluster_metrics.groupby("Cluster_Hierarchical")[
    "combined_score"
].rank(ascending=False)

# Step 4: Output the Rankings
for cluster_name, cluster_data in agent_cluster_metrics.groupby("Cluster_Hierarchical"):
    print(f"Top agents for cluster '{cluster_name}':")
    print(cluster_data.sort_values("rank").head(5))  # Top 5 agents per cluster


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import pandas as pd
import numpy as np

# Step 1: Combine agent features with client segment data
# Assuming `agent_df` contains agent features and `full_data` includes client clusters
numeric_columns = full_data.select_dtypes(include=["number"]).columns

# Step 2: Group by the Cluster_Hierarchical and aggregate numeric columns
agent_data = full_data.groupby("Cluster_Hierarchical")[numeric_columns].mean().reset_index()

agent_data = agent_df.copy()

# Merge agent data with cluster-level data
matching_data = agent_data.merge(agent_cluster_data, how="cross")

# Step 2: Define target and features
# Use `cnt_converted` or a similar proxy metric as the target
matching_data["success_metric"] = (
    matching_data["pct_inforce"] - (matching_data["pct_cancel"] + matching_data["pct_lapsed"])
)

# Define features for prediction
agent_features = [
    "agent_age", "agent_gender", "agent_marital", "agent_tenure",
    "pct_sx0_unknown", "pct_sx1_male", "pct_sx2_female",
    "pct_prod_0_cnvrt", "pct_prod_1_cnvrt", "pct_prod_2_cnvrt",  # Add all pct_prod_* columns
    "pct_ag01_lt20", "pct_ag02_20to24", "pct_ag03_25to29",  # Add all pct_ag* columns
]

client_features = ["household_size", "economic_status", "family_size", "clt_Age"]
features = agent_features + client_features

# Step 3: Train a model to match agents to clusters
X = matching_data[features]
y = (matching_data["success_metric"] > 0).astype(int)  # Binary classification: Success or not

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Step 4: Evaluate model
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

# Step 5: Predict and rank agents for each cluster
matching_data["predicted_success"] = model.predict_proba(X)[:, 1]

# Rank agents by predicted success for each cluster
ranked_agents = (
    matching_data.groupby("Cluster_Hierarchical")
    .apply(lambda x: x.sort_values("predicted_success", ascending=False))
    .reset_index(drop=True)
)

# Display top agents for each cluster
print(ranked_agents[["Cluster_Hierarchical", "agntnum", "predicted_success"]].head())

In [ ]:
# Create a binary target variable
full_data['agent_conversion_success'] = (full_data['cnt_converted'] > 0).astype(int)

# Step 1: Ensure 'agent_conversion_success' is the target variable
full_data['agent_conversion_success'] = full_data['agent_conversion_success'].astype(int)

# Step 2: Identify feature columns (excluding cluster and target)
feature_cols = full_data.select_dtypes(include=np.number).columns.tolist()
feature_cols = [col for col in feature_cols if col not in ["agent_conversion_success", "Cluster", "agntnum", "Cluster_Hierarchical", 'Cluster', 'Cluster_Hierarchical']]  # Remove target and cluster columns

# Step 3: Standardize numerical features
scaler = StandardScaler()
full_data[feature_cols] = scaler.fit_transform(full_data[feature_cols])

# Step 4: Train a logistic regression model for each cluster
cluster_models = {}
for cluster in full_data["Cluster_Hierarchical"].unique():
    print(f"Training Logistic Regression for Cluster: {cluster}")

    # Filter data for this cluster
    cluster_data = full_data[full_data["Cluster_Hierarchical"] == cluster]

    # Split into training & testing
    X_train, X_test, y_train, y_test = train_test_split(cluster_data[feature_cols],
                                                        cluster_data["agent_conversion_success"],
                                                        test_size=0.2, random_state=42)

    # Check if y_test has exactly two classes
    if len(np.unique(y_test)) != 2:
        print(f"Skipping cluster {cluster} because y_test has only one class: {np.unique(y_test)}")
        continue

    # Train logistic regression
    model = LogisticRegression()
    model.fit(X_train, y_train)

    # Store trained model
    cluster_models[cluster] = model

    # Evaluate
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)[:, 1]

    acc = accuracy_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_pred_proba)

    print(f"Accuracy: {acc:.4f}, AUC: {auc:.4f}")

# Step 5: Predict best agent for a new client
def predict_best_agent(client_features, client_cluster, agent_data):
    """
    Given a client’s features and cluster, predict the agent with the highest probability of conversion.

    :param client_features: DataFrame with client features (matching feature_cols)
    :param client_cluster: The client's assigned cluster
    :param agent_data: DataFrame with all agent features (matching feature_cols)
    :return: Best agent's ID and probability
    """
    model = cluster_models.get(client_cluster)
    if not model:
        raise ValueError(f"No model found for cluster: {client_cluster}")

    # Predict probabilities for each agent
    agent_probabilities = model.predict_proba(agent_data[feature_cols])[:, 1]

    # Find the best agent
    best_agent_idx = np.argmax(agent_probabilities)
    best_agent_id = agent_data.iloc[best_agent_idx]["agntnum"]
    best_agent_prob = agent_probabilities[best_agent_idx]

    return best_agent_id, best_agent_prob

